In [1]:
import numpy as np
from scipy.linalg import eig
from scipy.interpolate import CubicSpline
from scipy.optimize import minimize_scalar

import param
import holoviews as hv
import panel as pn
hv.extension('bokeh', logo=False)
pn.extension('katex')

<div style="height:2cm;">
<div style="float:center;width:100%;text-align:center;">
<strong style="height:100px;color:darkred;font-size:40px;">
The Generalized Eigenvalue Problem
</strong>
</div></div>

# 1. Motivation

In engineering and data science, we often compare two matrices acting on the same vector space.

The **standard eigenvalue problem** seeks $\lambda$ and $x \ne 0$ such that:

$$ A x = \lambda x. $$

The **generalized eigenvalue problem (GEP)** involves a second matrix $B$:

$$ A x = \lambda B x. $$

This describes directions where the action of $A$ is proportional to that of $B$.
It arises naturally in **modal analysis**, **discriminant analysis**, and **regularized problems**.

# 2. Matrix Pencil Formulation

We define the **matrix pencil**:

$$ L(\lambda) = A - \lambda B. $$

A scalar $\lambda$ is a **generalized eigenvalue** if:

$$ \det(A - \lambda B) = 0. $$

**Regular pencil:** $\det(A - \lambda B)$ is not identically zero.

**Singular pencil:** $\det(A - \lambda B) \equiv 0$. (More advanced — not covered here.)

# 3. The Generalized Rayleigh Quotient

For symmetric $A$, $B$, define:

$$ \rho(x) = \frac{x^T A x}{x^T B x}, \quad x^T B x \ne 0. $$

The **stationary values** of $\rho(x)$ correspond to the generalized eigenvalues $\lambda$, with $x$ the corresponding eigenvectors.

This leads to a variational formulation of GEP.

# 4. Variational Formulation

To find stationary points of $\rho(x)$, maximize or minimize it subject to $x^T B x = 1$:

$$ \max_x \; x^T A x \quad \text{subject to } x^T B x = 1. $$

Using Lagrange multipliers:

$$ \nabla_x (x^T A x - \lambda (x^T B x - 1)) = 0 \Rightarrow A x = \lambda B x. $$

In [6]:
class GEPRayleighViewer(pn.viewable.Viewer):
    alpha = param.Number(default=1.5, bounds=(0.1, 3.0), step=0.05, doc="Value for B[2,2] (α)")

    def __init__(self, A_fixed, B_template, **params):
        super().__init__(**params)
        self.A = A_fixed
        self.B_template = B_template

    @pn.depends('alpha')
    def _plot(self):
        B       = self.B_template.copy()
        B[1, 1] = self.alpha

        theta_rad = np.linspace(0, np.pi, 600)
        theta_deg = np.degrees(theta_rad)
        rhos = []
        for t in theta_rad:
            x = np.array([np.cos(t), np.sin(t)])
            num = x.T @ self.A @ x
            den = x.T @ B @ x
            rhos.append(num / den if den > 1e-12 else np.nan)

        spline = CubicSpline(theta_deg, rhos)
        res_min = minimize_scalar(spline, bounds=(0, 180), method='bounded')
        res_max = minimize_scalar(lambda t: -spline(t), bounds=(0, 180), method='bounded')
        theta_min, rho_min = res_min.x, res_min.fun
        theta_max, rho_max = res_max.x, -res_max.fun

        curve = hv.Curve((theta_deg, rhos), 'θ (°)', 'ρ(θ)').opts(width=600, height=350)
        hlines = [
            hv.HLine(rho_min).opts(color='red', line_dash='dashed'),
            hv.HLine(rho_max).opts(color='red', line_dash='dashed')
        ]
        vlines = [
            hv.VLine(theta_min).opts(color='blue', line_dash='dotted'),
            hv.VLine(theta_max).opts(color='blue', line_dash='dotted')
        ]

        return curve * hv.Overlay(hlines + vlines).opts(show_grid=True)

    def __panel__(self):
        return pn.Column(
            pn.Param(self, parameters=['alpha'], widgets={'alpha': pn.widgets.FloatSlider}),
            self._plot
        )


### Interactive Viewer: Generalized Rayleigh Quotient
We plot $\rho(\theta) = x^T A x / x^T B x$ where $x(\theta) = (\cos \theta, \sin \theta)^T$.
Red markers show generalized eigenvalues; vertical lines show stationary angles.

In [8]:
A = np.array([[3.0, 2.0], [1.0, 4.0]])
B_template = np.array([[2.0, 0.5], [0.1, 1.5]])
GEPRayleighViewer(A, B_template).servable()

Column
    [0] Param(GEPRayleighViewer, widgets={'alpha': <class '...})
    [1] ParamMethod(method, _pane=HoloViews, defer_load=False)

# 5. Summary

- The **GEP** $A x = \lambda B x$ describes how two matrices compare across directions.
- The **matrix pencil** $A - \lambda B$ defines eigenvalues via singularity.
- The **generalized Rayleigh quotient** characterizes these values variationally.
- The next notebook explores **applications** in vibration, control, and data science.